# Codealong Notebook

Use this notebook as your "scratch pad" as you go through the course contents. Feel free to copy any example code and tweak it to get a better understanding of how it works!

Use the **+** button or `Insert` menu to add additional code cells as needed.

In [1]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-32034342212667733781206696e1b8b3b604.60383220"

In [2]:
ukraine_prompt = """
Question: When did Russia invade Ukraine?
Answer: 
"""

ukraine_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=ukraine_prompt
)

InvalidRequestError: Insufficient budget available

In [ ]:
#response type
'''<OpenAIObject text_completion id=cmpl-6hgvbhgyasdfkj at 0x7f765af9d8b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "Russia invaded Ukraine in 2014."
    }
  ],
  "created": 1679090273,
  "id": "cmpl-6hgvbhgyasdfkj",
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 6,
    "prompt_tokens": 14,
    "total_tokens": 20
  }
}'''

In [ ]:
ukraine_answer["choices"][0]["text"]

## Step 1

### Loading the Data with `pandas`

In [ ]:
import requests

# Base endpoint URL for the Wikipedia API
endpoint = "https://en.wikipedia.org/w/api.php"

# Set the parameters for the API request
params = {
    "action": "query",
    "format": "json",
    "prop": "extracts",
    "titles": "2022",
    "explaintext": True,
    "exsectionformat": "plain",
    "exlimit": "max"
}

# Make the request to the Wikipedia API
response = requests.get(endpoint, params=params)
data = response.json()

# Extract and print the response data
print(data)


In [ ]:
import pandas as pd

# Extract the page content
pages = data['query']['pages']
content = ""
for page_id, page_info in pages.items():
    content = page_info['extract']

# Process the content to extract events
events = []
for line in content.split('\n'):
    if line.startswith('* '):  # Each event might be listed with a bullet point
        events.append(line[2:])

# Convert to a pandas DataFrame for better readability
events_df = pd.DataFrame(events, columns=["Event"])
print(events_df.head(10))  # Display the first 10 events

### Creating an Embeddings Index with `openai.Embedding`

In [ ]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
# Generate the embedding response
sentences =[]
for sentence in df.text.values(:)
    response = openai.Embedding.create(
    engine = EMBEDDING_MODEL_NAME,
    input = USER_QUESTION
    )
    sentences.append(response)
print(response.keys())

In [ ]:
#embeddings for dataframe
df["embeddings"] = sentences
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
df

In [ ]:
USER_QUESTION = ukraine_answer

# Extract the embeddings from the response
question_embeddings = response["data"][0]["embedding"]

#print(question_embeddings[:100])

## Step 2

### Finding Relevant Data with Cosine Similarity

In [ ]:
from openai.embeddings_utils import distances_from_embeddings

# Create a list containing the distances from question_embeddings
distances = distances_from_embeddings(question_embeddings, df["embeddings"])

In [ ]:
df["distances"] = distances
df.sort_values(by="distances", ascending=True, inplace=True)
df.head(5)

## Step 3

### Tokenizing with `tiktoken`

In [ ]:
import tiktoken

### Composing a Custom Text Prompt

In [ ]:
def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
                Answer the question based on the context below, and if the question
                can't be answered based on the context, say "I don't know"

                Context: 

                {}

                ---

                Question: {}
                Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [ ]:
custom_prompt = create_prompt(USER_QUESTION, df, 4097)

## Step 4

### Getting a Custom Q&A Response with `openai.Completion`

In [ ]:
# Generic example code (not needed for this tutorial)
COMPLETION_MODEL_NAME="gpt-3.5-turbo-instruct",
max_answer_tokens=150

try:
    response = openai.Completion.create(
        model=COMPLETION_MODEL_NAME,
        prompt=custom_prompt,
        max_tokens=max_answer_tokens
    )
    return response["choices"][0]["text"].strip()
except Exception as e:
    print(e)
    return ""